# Computer Vision HackPack
In this Hackpack we will be creating an application that recognises objects from a video stream!

### Prerequisites
* Python 3.8 *Later versions dont play nice with the Simple Online Realtime Tracking (SORT) module*
    * You can get this easily by creating a virtual environment. Think of this as a sandbox where you can install python packages in isolation of other ones that may be stored on your computer.
      * To do this install miniconda https://docs.anaconda.com/miniconda/miniconda-install/
      * Create the environment by ruuning the command *conda create -n hackpack python=3.8 anaconda*
      * Activate the environment by running *conda activate hackpack*   
* Install the required packages by running *pip install opencv-python*, *pip install numpy* and then *pip install sort-track*
* We also require the model, You Only Look Once (YOLO), we will use to detect objects! You can find the models folder here. Place it in the same folder as this notebook.
#INSERT LINK HERE


## Import the packages you just installed

In [1]:
import cv2
import numpy as np
# from sort.tracker import SortTracker
import time

ModuleNotFoundError: No module named 'cv2'

## Load in the model

In [3]:
# Load YOLO
net = cv2.dnn.readNet('./models/yolov3.weights', './models/yolov3.cfg')
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
classes = ["person"]  

## Initialise the SORT Tracker.
This model compares the positions of objects between frames and tries to guess whther they are the same or different objects. If they are different we will draw a new bounding box around it with a new id. If we guess that it's the same then we move the bounding box with that ID from the last frame to surround the object at its new location.

In [4]:
tracker = SortTracker()

## Start the video capture
Here we use OpenCV2 to start a video stream

In [ ]:
cap = cv2.VideoCapture(0) # ID of the camera being used, try and change this if it's not working or you have more than one camera
width  = cap.get(3)  
height = cap.get(4)  

# Hot loop 🔥🔥

This is where the bulk of the processing happens but in summary it:
<ol>
<li> Captures a frame
<li> Puts it through the YOLO model
<li> For each object detected. If we are sure enough that it exists then we work out a Bounding Box for it
<li> We pass the Bounding Boxes to the SORT module which determines if its a new object
<li> We pass the boxes along with their IDS to be drawn on the next frame
</ol>
Hold q to stop the capture!

In [ ]:
while True:
    # Capture frame-by-frame 1

    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Convert the frame to a blob and pass through the network 2
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

   
    class_ids = []
    confidences = []
    boxes = []
    

    # Analyze the outs array 3
    for out in outs:
        for detection in out:
            
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Convert YOLO detections to SORT format (x, y, w, h) 4
    detections = []
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            detections.append([x, y, x + w, y + h,1,1])

    # Update SORT tracker with current frame detections 4
    try:
        track_bbs_ids = tracker.update(np.array(detections),"")
    except:
        pass
    
    

    # Drawing tracked objects on the image 5
    items = 0
    for track in track_bbs_ids:
    
        x, y, w, h, track_id,_,_ = track
        cv2.rectangle(frame, (int(x), int(y)), (int(w), int(h)), (255, 0, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (int(x), int(y - 10)), 0, 0.5, (255, 0, 0), 2)

    # Display the resulting frame 5
    cv2.imshow('Frame', frame)
    
    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()

